GOAL1: Find if the size of the city have an impact on what people are voting for in the anticipated legislative election of 2024

Importation of the librairies for this EDA project

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy as sp
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')

from pyspark.sql.functions import regexp_replace, col, when

from pyspark.sql.types import FloatType, IntegerType


This is a simple EDA project with data coming from the official french government website about the results of the legislative elections from 2002 to 2024
Source of the files: https://www.data.gouv.fr/fr/pages/donnees-des-elections/

We will use PySpark for to handle the csv file of the year 2024

Tuto for the installation of Pyspark: https://www.datacamp.com/tutorial/pyspark-tutorial-getting-started-with-pyspark

Create the Pyspark session

In [2]:
spark = SparkSession.builder.appName("Pyspark Election Project").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()

Read the csv file for the election of 2024 with options:
 - delimiter as ";" because we use a CSV file
 - keep the Header of the file 

In [3]:
df_res_2024_communes = spark.read.option("delimiter", ";").option("header", True).csv("../Data/2024/resultats-definitifs-par-commune.csv")

Show the first 10 rows of the dataset

In [4]:
df_res_2024_communes.show(10)

+----------------+-------------------+------------+--------------------+--------+-------+---------+-----------+-------------+--------+-------------------+------------------+------+-----------------+----------------+----+---------------+--------------+-------------------+-----------------+--------------+-----------------+---------------+------+-----------------+-----------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+-----------------+-----------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+-----------------+-----------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+-----------------+-----------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+-----------------+-----------------+-----+-------------------+-----------------+--------------+----------------

In [5]:
# Replace spaces with underscores in column names
for name in df_res_2024_communes.columns:
    df_res_2024_communes = df_res_2024_communes.withColumnRenamed(name, name.replace(" ", "_"))


In [6]:
# Replace spaces with underscores in column names
for name in df_res_2024_communes.columns:
    df_res_2024_communes = df_res_2024_communes.withColumnRenamed(name, name.replace("%", "Pourcentage"))

In [7]:
df_res_2024_communes.show(10)

+----------------+-------------------+------------+--------------------+--------+-------+-------------------+-----------+-----------------------+--------+-----------------------------+----------------------------+------+---------------------------+--------------------------+----+-------------------------+------------------------+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+-------

In [8]:
cols = len(df_res_2024_communes.columns)
print(f"DataFrame Columns count : {cols}")

DataFrame Columns count : 180


The dataset has 180 different columns with unique and repeated informations for different candidates:
Unique columns:
- Code_département
- Libellé_département
- Code_commune
- Libellé_commune
- Inscrits
- Votants
- Pourcentage_Votants
- Abstentions
- Pourcentage_Abstentions
- Exprimés
- Pourcentage_Exprimés/inscrits
- Pourcentage_Exprimés/votants
- Blancs
- Pourcentage_Blancs/inscrits
- Pourcentage_Blancs/votants
- Nuls
- Pourcentage_Nuls/inscrits
- Pourcentage_Nuls/votants

Repeated columns (X being a value between 1 and 18):

- Numéro_de_panneau X
- Nuance_candidat X
- Nom_candidat X
- Prénom_candidat X
- Sexe_candidat X
- Voix_X
- Pourcentage_Voix/inscrits X
- Pourcentage_Voix/exprimés X
- Elu_X

If you count the number of unique values and you add the repetition of the repeated columns you get:\
18 + (9*18) = 180 columns like expected 

Display the schema of the DataFrame

Remarks:
- All the fields of the schema are of the type string
- All the fields of the schema are nullable

We will have to transform some types of the values in the schema

In [9]:
# Print the schema of the DataFrame
df_res_2024_communes.printSchema()


root
 |-- Code_département: string (nullable = true)
 |-- Libellé_département: string (nullable = true)
 |-- Code_commune: string (nullable = true)
 |-- Libellé_commune: string (nullable = true)
 |-- Inscrits: string (nullable = true)
 |-- Votants: string (nullable = true)
 |-- Pourcentage_Votants: string (nullable = true)
 |-- Abstentions: string (nullable = true)
 |-- Pourcentage_Abstentions: string (nullable = true)
 |-- Exprimés: string (nullable = true)
 |-- Pourcentage_Exprimés/inscrits: string (nullable = true)
 |-- Pourcentage_Exprimés/votants: string (nullable = true)
 |-- Blancs: string (nullable = true)
 |-- Pourcentage_Blancs/inscrits: string (nullable = true)
 |-- Pourcentage_Blancs/votants: string (nullable = true)
 |-- Nuls: string (nullable = true)
 |-- Pourcentage_Nuls/inscrits: string (nullable = true)
 |-- Pourcentage_Nuls/votants: string (nullable = true)
 |-- Numéro_de_panneau_1: string (nullable = true)
 |-- Nuance_candidat_1: string (nullable = true)
 |-- Nom_can

Number of rows

In [10]:
df_res_2024_communes.count()

31392

Create the list of the features that needs to be transformed into int

In [11]:
integer_features = {
    "Code_département": IntegerType(),
    "Code_commune": IntegerType(),
    "Inscrits": IntegerType(),
    "Votants": IntegerType(),
    "Abstentions": IntegerType(),
    "Exprimés":IntegerType(),
    "Blancs": IntegerType(),
    "Nuls": IntegerType(), 
}

Update the integer features dictionnary to add the repeated columns which need to be updated

In [12]:
for i in range(1, 19, 1):
    integer_features[f"Numéro_de_panneau_{i}"] = IntegerType()
    integer_features[f"Voix_{i}"] = IntegerType()

In [13]:
integer_features

{'Code_département': IntegerType(),
 'Code_commune': IntegerType(),
 'Inscrits': IntegerType(),
 'Votants': IntegerType(),
 'Abstentions': IntegerType(),
 'Exprimés': IntegerType(),
 'Blancs': IntegerType(),
 'Nuls': IntegerType(),
 'Numéro_de_panneau_1': IntegerType(),
 'Voix_1': IntegerType(),
 'Numéro_de_panneau_2': IntegerType(),
 'Voix_2': IntegerType(),
 'Numéro_de_panneau_3': IntegerType(),
 'Voix_3': IntegerType(),
 'Numéro_de_panneau_4': IntegerType(),
 'Voix_4': IntegerType(),
 'Numéro_de_panneau_5': IntegerType(),
 'Voix_5': IntegerType(),
 'Numéro_de_panneau_6': IntegerType(),
 'Voix_6': IntegerType(),
 'Numéro_de_panneau_7': IntegerType(),
 'Voix_7': IntegerType(),
 'Numéro_de_panneau_8': IntegerType(),
 'Voix_8': IntegerType(),
 'Numéro_de_panneau_9': IntegerType(),
 'Voix_9': IntegerType(),
 'Numéro_de_panneau_10': IntegerType(),
 'Voix_10': IntegerType(),
 'Numéro_de_panneau_11': IntegerType(),
 'Voix_11': IntegerType(),
 'Numéro_de_panneau_12': IntegerType(),
 'Voix_12

Change the types of all the features within the column

In [14]:
for features, types in integer_features.items():
    df_res_2024_communes = df_res_2024_communes.withColumn(features, df_res_2024_communes[features].cast(types))

Change the types of the float values

In [15]:
float_features = {
    "Pourcentage_Votants": FloatType(),
    "Pourcentage_Abstentions": FloatType(),
    "Pourcentage_Exprimés/inscrits": FloatType(),
    "Pourcentage_Exprimés/votants": FloatType(),
    "Pourcentage_Blancs/inscrits": FloatType(),
    "Pourcentage_Blancs/votants": FloatType(),
    "Pourcentage_Nuls/inscrits": FloatType(),
    "Pourcentage_Nuls/votants": FloatType(),
}

In [16]:
for i in range(1, 19, 1):
    float_features[f"Pourcentage_Voix/inscrits_{i}"] = FloatType()
    float_features[f"Pourcentage_Voix/exprimés_{i}"] = FloatType()

In [17]:
float_features

{'Pourcentage_Votants': FloatType(),
 'Pourcentage_Abstentions': FloatType(),
 'Pourcentage_Exprimés/inscrits': FloatType(),
 'Pourcentage_Exprimés/votants': FloatType(),
 'Pourcentage_Blancs/inscrits': FloatType(),
 'Pourcentage_Blancs/votants': FloatType(),
 'Pourcentage_Nuls/inscrits': FloatType(),
 'Pourcentage_Nuls/votants': FloatType(),
 'Pourcentage_Voix/inscrits_1': FloatType(),
 'Pourcentage_Voix/exprimés_1': FloatType(),
 'Pourcentage_Voix/inscrits_2': FloatType(),
 'Pourcentage_Voix/exprimés_2': FloatType(),
 'Pourcentage_Voix/inscrits_3': FloatType(),
 'Pourcentage_Voix/exprimés_3': FloatType(),
 'Pourcentage_Voix/inscrits_4': FloatType(),
 'Pourcentage_Voix/exprimés_4': FloatType(),
 'Pourcentage_Voix/inscrits_5': FloatType(),
 'Pourcentage_Voix/exprimés_5': FloatType(),
 'Pourcentage_Voix/inscrits_6': FloatType(),
 'Pourcentage_Voix/exprimés_6': FloatType(),
 'Pourcentage_Voix/inscrits_7': FloatType(),
 'Pourcentage_Voix/exprimés_7': FloatType(),
 'Pourcentage_Voix/inscri

Clean the pourcentage features that have the string type

In [18]:
for features, types in float_features.items():

    df_res_2024_communes = df_res_2024_communes.withColumn(
        features,
        regexp_replace(col(features), "[^0-9,]", "")  # Remove all other characters that digits and ,
    ).withColumn(
        features,
        regexp_replace(col(features), ",", ".")
    ).withColumn(
        features,
        col(features).cast(types)
    )

In [19]:
# Updates schema
df_res_2024_communes.printSchema()

root
 |-- Code_département: integer (nullable = true)
 |-- Libellé_département: string (nullable = true)
 |-- Code_commune: integer (nullable = true)
 |-- Libellé_commune: string (nullable = true)
 |-- Inscrits: integer (nullable = true)
 |-- Votants: integer (nullable = true)
 |-- Pourcentage_Votants: float (nullable = true)
 |-- Abstentions: integer (nullable = true)
 |-- Pourcentage_Abstentions: float (nullable = true)
 |-- Exprimés: integer (nullable = true)
 |-- Pourcentage_Exprimés/inscrits: float (nullable = true)
 |-- Pourcentage_Exprimés/votants: float (nullable = true)
 |-- Blancs: integer (nullable = true)
 |-- Pourcentage_Blancs/inscrits: float (nullable = true)
 |-- Pourcentage_Blancs/votants: float (nullable = true)
 |-- Nuls: integer (nullable = true)
 |-- Pourcentage_Nuls/inscrits: float (nullable = true)
 |-- Pourcentage_Nuls/votants: float (nullable = true)
 |-- Numéro_de_panneau_1: integer (nullable = true)
 |-- Nuance_candidat_1: string (nullable = true)
 |-- Nom_ca

In [20]:
df_res_2024_communes.show()

+----------------+-------------------+------------+--------------------+--------+-------+-------------------+-----------+-----------------------+--------+-----------------------------+----------------------------+------+---------------------------+--------------------------+----+-------------------------+------------------------+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+-------

We still have the Elu columns to handle\
We will transform them into boolean values as élu is True and the NULL values are False

In [21]:
# Create the list of the Elu column' names
elu_columns = [f"Elu_{i}" for i in range(1, 19, 1)]

In [22]:
for columns in elu_columns:
    df_res_2024_communes = df_res_2024_communes.withColumn(
    columns, when(col(columns) == "élu", True).otherwise(False)
)

Count the number of columns with Null values in a specific column

In [23]:
from pyspark.sql.functions import isnull, sum, col, when

null_count_df = df_res_2024_communes.select(
    sum(when(isnull(col("Code_commune")), 1).otherwise(0)).alias("null_count")
)

null_count_df.show()  # Display the result

+----------+
|null_count|
+----------+
|       575|
+----------+



In [24]:
from pyspark.sql.functions import isnull, sum, col, when

null_count_df = df_res_2024_communes.select(
    sum(when(isnull(col("Code_département")), 1).otherwise(0)).alias("null_count")
)

null_count_df.show()  # Display the result

+----------+
|null_count|
+----------+
|       575|
+----------+



We have to handle this missing values for the Code_Commune and Code_departement features\
Missing values:
 -  Code_Commune: 575
 -  Code_département: 575

In [34]:
# Create a dictionnary with the missing departement codes:
Code_departement = {
    "Corse-du-Sud": '2A',
    "Haute-Corse": '2B',
    "Français établis hors de France": 99,
    "Saint-Barthélemy": 977, 
    "Saint-Martin": 978, 
}

# For Saint-Barthélemy and Saint-Martin we will base the condition on the Libellé_commune feature
# For the other missing departements we will base the condition on the Libellé_département feature


In [32]:
for key, value in Code_departement.items():
    df_res_2024_communes = df_res_2024_communes.withColumn(
        "Code_département",
        when(col("Libellé_département") == key, value).otherwise(col("Code_département"))
    )

In [35]:
for key, value in Code_departement.items():
    df_res_2024_communes = df_res_2024_communes.withColumn(
        "Code_département",
        when(col("Libellé_commune") == key, value).otherwise(col("Code_département"))
    )

In [36]:
from pyspark.sql.functions import isnull, sum, col, when

null_count_df = df_res_2024_communes.select(
    sum(when(isnull(col("Code_département")), 1).otherwise(0)).alias("null_count")
)

null_count_df.show()  # Display the result

+----------+
|null_count|
+----------+
|         0|
+----------+



In [37]:
# Assuming your DataFrame is named df_res_2024_communes
target_libelle_departement = "Haute-Corse" 

df_res_2024_communes.filter(col("Libellé_département") == target_libelle_departement) \
                     .select("Code_département") \
                     .show()

+----------------+
|Code_département|
+----------------+
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
|              2B|
+----------------+
only showing top 20 rows



In [ ]:
# Updates schema
df_res_2024_communes.printSchema()

In [ ]:
print(df_res_2024_communes.dtypes)

Convert the DataFrame in Pandas

In [29]:
pd_df = df_res_2024_communes.select("*").toPandas()

In [ ]:
type(pd_df)

In [ ]:
print(pd_df["Code_département"])

In [ ]:
df.dtypes

Write the DataFrame into csv

In [30]:
pd_df.to_csv("../Clean_Data/2024/clean_dataset_legislative_2024.csv", sep=";", header=True, index=False)

Read the csv file

In [ ]:
df = pd.read_csv("../Clean_Data/2024/clean_dataset_legislative_2024.csv", sep=";")

In [ ]:
df.head(5)

In [ ]:
df.Code_département.dtype